# More Earth Engine: Irrigation, Snow, and Soils Data Extraction

In this notebook, we will:
1. Use the `sample_snodas_swe` function to perform snodas snow water equivalent (SWE) extraction.
2. Use the `get_irrigation` function to perform irrigation mask extraction.
3. Use the `get_ssurgo` function to perform soils extraction.

Ensure that the Earth Engine Python API is authenticated and configured, and that the Earth Engine CLI is available in your environment.

## 1. Import Libraries and Authorize Earth Engine

In [1]:
import os
import sys
import ee

# append the project path to the environment (mine is in /home/dgketchum/PycharmProjects')
sys.path.append('/home/dgketchum/PycharmProjects/swim-rs')

from data_extraction.ee.snodas_export import sample_snodas_swe
from data_extraction.ee.ee_props import get_irrigation, get_ssurgo
from data_extraction.ee.ee_utils import is_authorized
from utils.google_bucket import list_and_copy_gcs_bucket

sys.path.insert(0, os.path.abspath('../..'))
sys.setrecursionlimit(5000)

if not is_authorized():
    ee.Authenticate()
ee.Initialize()

Authorized


## 1: Extract Raster Data

Here, we're going to use functions with a similar workflow to those we used to get ETf and NDVI data from Earth Engine.

For the raster data extraction, there are three functions we need to run:

*   **`sample_snodas_swe`**:
    *   This function iterates over the daily SNODAS images in Earth Engine, extracting mean SWE for each sample plot for each day, September through May.
*   **`get_irrigation`**:
    *   This function uses IrrMapper to get statistics about the irrigation status of each plot for each year, including the fraction of the plot that was irrigated.
*   **`get_ssurgo`**:
    *   This function uses data summarized and put in a public Earth Engine asset by Charles Morton at Desert Research Institute from SSURGO (https://www.nrcs.usda.gov/resources/data-and-reports/soil-survey-geographic-database-ssurgo) to summarize plot-scale soil texture and hydraulic properties used by SWIM-RS.

In [2]:
ee_account = 'ee-dgketchum'

# Define Constants and Remote Sensing Data Paths
# TODO: remove hard-coded collecitons and use variables defined here
SWE = 'projects/earthengine-legacy/assets/projects/climate-engine/snodas/daily'


root = '/media/research/IrrigationGIS/swim'
bucket = 'wudr'
fields = 'projects/ee-dgketchum/assets/swim/mt_sid_boulder'

command = '/home/dgketchum/google-cloud-sdk/bin/gsutil'

# We must specify which column in the shapefile represents the field's unique ID, in this case it is 'FID_1'
FEATURE_ID = 'FID_1'

### SWE Data

Notes
- Ensure that your Cloud Storage bucket has the correct permissions for Earth Engine to write to it.
- This will produce a monthly dataset for Sep - May, regardless of SWE status at the sample plots.

In [3]:
# Here, we run the sample_snodas_swe function on the uploaded asset.

chk = os.path.join(root, 'examples/tutorial/landsat/extracts/swe')
sample_snodas_swe(fields, bucket, debug=False, check_dir=None, feature_id=FEATURE_ID)


swe_2004_01
swe_2004_02
swe_2004_03
swe_2004_04
swe_2004_05
swe_2004_06
swe_2004_07
swe_2004_08
swe_2004_09
swe_2004_10
swe_2004_11
swe_2004_12
swe_2005_01
swe_2005_02
swe_2005_03
swe_2005_04
swe_2005_05
swe_2005_06
swe_2005_07
swe_2005_08
swe_2005_09
swe_2005_10
swe_2005_11
swe_2005_12
swe_2006_01
swe_2006_02
swe_2006_03
swe_2006_04
swe_2006_05
swe_2006_06
swe_2006_07
swe_2006_08
swe_2006_09
swe_2006_10
swe_2006_11
swe_2006_12
swe_2007_01
swe_2007_02
swe_2007_03
swe_2007_04
swe_2007_05
swe_2007_06
swe_2007_07
swe_2007_08
swe_2007_09
swe_2007_10
swe_2007_11
swe_2007_12
swe_2008_01
swe_2008_02
swe_2008_03
swe_2008_04
swe_2008_05
swe_2008_06
swe_2008_07
swe_2008_08
swe_2008_09
swe_2008_10
swe_2008_11
swe_2008_12
swe_2009_01
swe_2009_02
swe_2009_03
swe_2009_04
swe_2009_05
swe_2009_06
swe_2009_07
swe_2009_08
swe_2009_09
swe_2009_10
swe_2009_11
swe_2009_12
swe_2010_01
swe_2010_02
swe_2010_03
swe_2010_04
swe_2010_05
swe_2010_06
swe_2010_07
swe_2010_08
swe_2010_09
swe_2010_10
swe_2010_11
swe_

Copy the data to the local machine once the processing is complete, as with the remote sensing data.

In [5]:
dst = os.path.join(root, 'examples/tutorial/snodas/extracts')
glob_ = 'swe'
list_and_copy_gcs_bucket(command, bucket, dst, glob=glob_, dry_run=False)


Copying gs://wudr/swe_2004_01.csv: Copying gs://wudr/swe_2004_01.csv...
\ [1 files][ 49.9 KiB/ 49.9 KiB]                                                
Operation completed over 1 objects/49.9 KiB.                                     

Copied gs://wudr/swe_2004_01.csv to /media/research/IrrigationGIS/swim/examples/tutorial/snodas/extracts
Copying gs://wudr/swe_2004_02.csv: Copying gs://wudr/swe_2004_02.csv...
/ [1 files][ 41.5 KiB/ 41.5 KiB]                                                
Operation completed over 1 objects/41.5 KiB.                                     

Copied gs://wudr/swe_2004_02.csv to /media/research/IrrigationGIS/swim/examples/tutorial/snodas/extracts
Copying gs://wudr/swe_2004_03.csv: Copying gs://wudr/swe_2004_03.csv...
/ [1 files][ 14.9 KiB/ 14.9 KiB]                                                
Operation completed over 1 objects/14.9 KiB.                                     

Copied gs://wudr/swe_2004_03.csv to /media/research/IrrigationGIS/swim/examples/tu

KeyboardInterrupt: 

### Irrigation Data

Notes
- This will produce an annual dataset of the IrrMapper-estimated irrigated fraction for each sample plot.

In [5]:
description = 'tutorial_irr'
get_irrigation(fields, description, debug=False, selector=FEATURE_ID)

In [6]:
dst = os.path.join(root, 'examples/tutorial/properties')
list_and_copy_gcs_bucket(command, bucket, dst, glob=description, dry_run=False)

Copying gs://wudr/tutorial_irr.csv: Copying gs://wudr/tutorial_irr.csv...
/ [1 files][ 47.8 KiB/ 47.8 KiB]                                                
Operation completed over 1 objects/47.8 KiB.                                     

Copied gs://wudr/tutorial_irr.csv to /media/research/IrrigationGIS/swim/examples/tutorial/properties


### Soils Data

Notes
- This will produce an single dataset of the SSURGO-estimated soil properties fraction for each sample plot.

In [7]:
description = 'tutorial_ssurgo'
get_ssurgo(fields, description, debug=False, selector=FEATURE_ID)

tutorial_ssurgo


In [9]:
dst = os.path.join(root, 'examples/tutorial/properties')
list_and_copy_gcs_bucket(command, bucket, dst, glob=description, dry_run=False)

Copying gs://wudr/tutorial_ssurgo.csv: Copying gs://wudr/tutorial_ssurgo.csv...
/ [1 files][  6.7 KiB/  6.7 KiB]                                                
Operation completed over 1 objects/6.7 KiB.                                      

Copied gs://wudr/tutorial_ssurgo.csv to /media/research/IrrigationGIS/swim/examples/tutorial/properties
